In [1]:
'''Code for fine-tuning Inception V3 for a new task.

Start with Inception V3 network, not including last fully connected layers.

Train a simple fully connected layer on top of these.


'''

import numpy as np
import pandas as pd
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Flatten, Dropout
import keras.applications.inception_v3 as inception
from sklearn.preprocessing import MultiLabelBinarizer

IMSIZE = (299, 299)

PHOTO_PATH = r"\\devmachine\e$\data\katkam-secret-location\katkam-scaled"



Using TensorFlow backend.


In [2]:
cleaned_data = pd.read_csv("cleaned_data.csv")
weather_array = list(map(lambda x:x.split(',') ,cleaned_data["Mapped"].values))
Y = MultiLabelBinarizer().fit_transform(weather_array)
cleaned_data["Y"] = list(Y)
train_test_mask = np.random.rand(len(cleaned_data)) < 0.8
train_data = cleaned_data[train_test_mask]
test_data = cleaned_data[~train_test_mask]
# cleaned_data

In [3]:
# Start with an Inception V3 model, not including the final softmax layer.
base_model = inception.InceptionV3(weights='imagenet')
print ('Loaded Inception model')
for layer in base_model.layers:
    print(layer.name)

Loaded Inception model
input_1
conv2d_1
batch_normalization_1
activation_1
conv2d_2
batch_normalization_2
activation_2
conv2d_3
batch_normalization_3
activation_3
max_pooling2d_1
conv2d_4
batch_normalization_4
activation_4
conv2d_5
batch_normalization_5
activation_5
max_pooling2d_2
conv2d_9
batch_normalization_9
activation_9
conv2d_7
conv2d_10
batch_normalization_7
batch_normalization_10
activation_7
activation_10
average_pooling2d_1
conv2d_6
conv2d_8
conv2d_11
conv2d_12
batch_normalization_6
batch_normalization_8
batch_normalization_11
batch_normalization_12
activation_6
activation_8
activation_11
activation_12
mixed0
conv2d_16
batch_normalization_16
activation_16
conv2d_14
conv2d_17
batch_normalization_14
batch_normalization_17
activation_14
activation_17
average_pooling2d_2
conv2d_13
conv2d_15
conv2d_18
conv2d_19
batch_normalization_13
batch_normalization_15
batch_normalization_18
batch_normalization_19
activation_13
activation_15
activation_18
activation_19
mixed1
conv2d_23
batch_n

In [4]:
# Turn off training on base model layers
for layer in base_model.layers:
    layer.trainable = False
# k=base_model.get_layer('flatten').output
# Add on new fully connected layers for the output classes.
x = Dense(1024, activation='relu')(base_model.get_layer('avg_pool').output)
x = Dropout(0.5)(x)
predictions = Dense(5, activation='sigmoid', name='predictions')(x)

model = Model(inputs=base_model.input, outputs=predictions)

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


# Show some debug output
print (model.summary())

print ('Trainable weights')
print (model.trainable_weights)

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 299, 299, 3)   0                                            
____________________________________________________________________________________________________
conv2d_1 (Conv2D)                (None, 149, 149, 32)  864         input_1[0][0]                    
____________________________________________________________________________________________________
batch_normalization_1 (BatchNorm (None, 149, 149, 32)  96          conv2d_1[0][0]                   
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 149, 149, 32)  0           batch_normalization_1[0][0]      
___________________________________________________________________________________________

In [ ]:

#print(val_files)
#np.random.seed()
img_datagen = ImageDataGenerator()

def my_load_img(img_path,img_datagen,size):
    img = image.load_img(img_path, target_size=size)
    x = image.img_to_array(img)
    x = img_datagen.random_transform(x)
    x = img_datagen.standardize(x)
    return x
def my_img_generator(df,img_datagen,batch_size):
#     index_array = np.random.permutation(len(files))
    
    index = 0
    count = 0
    img_datas=[]
    img_labels=[]
    while 1:
        # create numpy arrays of input data
        # and labels, from each line in the file
        item = df.iloc[index]
        if count < batch_size:
            img_datas.append(my_load_img(PHOTO_PATH+"/"+item["Filename"],img_datagen,IMSIZE))
            img_labels.append(item["Y"])
            index=(index+1)%df.shape[0]
            count+=1
        else:
            count=0
            yield (np.array(img_datas),np.array(img_labels))
                # yield (np.array(img_datas),np.array(img_bboxes))
#             else:
#                 yield (np.array(img_datas),np.array(one_hot_labels))
            img_datas = []
            img_labels = []
            img_bboxes=[]
#             random.shuffle(files)
            

batch_size=32
# t = next(my_img_generator(train_files,img_datagen,batch_size))

# model.load_weights('catdog_pretrain_nf.h5') 
# train_data
# train_data.shape
my_train_generator = my_img_generator(train_data,img_datagen,batch_size)
my_test_generator = my_img_generator(test_data,img_datagen,batch_size)

# train_datagen = ImageDataGenerator(rescale=1./255)
# train_generator = train_datagen.flow_from_directory(
#         r'E:\workshare\Mind\A3\data\catdog',  # this is the target directory
#         target_size=IMSIZE,  # all images will be resized to 299x299 Inception V3 input
#         batch_size=batch_size,
#         class_mode='categorical')

#test_datagen = ImageDataGenerator(rescale=1./255)
# test_generator = test_datagen.flow_from_directory(
#         test_dir,  # this is the target directory
#         target_size=IMSIZE,  # all images will be resized to 299x299 Inception V3 input
#         batch_size=batch_size,
#         class_mode='categorical')
# train_data.shape
# print(next(train_generator)[1])
# print(next(my_train_generator)[1])

# print(a[1].shape)
# print(a[1])

In [ ]:

model.load_weights('weather.h5')
for i in range(20):
    model.fit_generator(
            my_train_generator,
            steps_per_epoch = 20,
            epochs = 10,
            validation_data = my_test_generator,
            verbose = 2,
            validation_steps = 5)
    model.save_weights('weather.h5') 


Epoch 1/10
14s - loss: 0.4430 - acc: 0.8284 - val_loss: 0.3565 - val_acc: 0.8487
Epoch 2/10
9s - loss: 0.4375 - acc: 0.8437 - val_loss: 0.3928 - val_acc: 0.8550
Epoch 3/10
9s - loss: 0.3327 - acc: 0.8553 - val_loss: 0.3591 - val_acc: 0.8725
Epoch 4/10
9s - loss: 0.3292 - acc: 0.8591 - val_loss: 0.4403 - val_acc: 0.8213
Epoch 5/10
9s - loss: 0.3567 - acc: 0.8491 - val_loss: 0.4490 - val_acc: 0.8287
Epoch 6/10
9s - loss: 0.3592 - acc: 0.8566 - val_loss: 0.7531 - val_acc: 0.7087
Epoch 7/10
9s - loss: 0.4424 - acc: 0.8312 - val_loss: 0.4574 - val_acc: 0.8388
Epoch 8/10
9s - loss: 0.4189 - acc: 0.8253 - val_loss: 0.4072 - val_acc: 0.8400
Epoch 9/10
9s - loss: 0.3827 - acc: 0.8275 - val_loss: 0.5706 - val_acc: 0.7150
Epoch 10/10
9s - loss: 0.4188 - acc: 0.8338 - val_loss: 0.3414 - val_acc: 0.8350
Epoch 1/10
9s - loss: 0.3040 - acc: 0.8669 - val_loss: 0.4091 - val_acc: 0.8487
Epoch 2/10
9s - loss: 0.3624 - acc: 0.8428 - val_loss: 0.6050 - val_acc: 0.7575
Epoch 3/10
9s - loss: 0.4022 - acc: 0.

9s - loss: 0.2914 - acc: 0.8688 - val_loss: 0.2951 - val_acc: 0.8763
Epoch 5/10
9s - loss: 0.3761 - acc: 0.8303 - val_loss: 0.2756 - val_acc: 0.9113
Epoch 6/10
9s - loss: 0.2930 - acc: 0.8741 - val_loss: 0.2629 - val_acc: 0.9037
Epoch 7/10
9s - loss: 0.3294 - acc: 0.8631 - val_loss: 0.2637 - val_acc: 0.8937
Epoch 8/10
9s - loss: 0.3379 - acc: 0.8550 - val_loss: 0.3085 - val_acc: 0.8875
Epoch 9/10
9s - loss: 0.3058 - acc: 0.8725 - val_loss: 0.2637 - val_acc: 0.9075
Epoch 10/10
9s - loss: 0.3531 - acc: 0.8391 - val_loss: 0.2537 - val_acc: 0.9163
Epoch 1/10
9s - loss: 0.3103 - acc: 0.8597 - val_loss: 0.2901 - val_acc: 0.8763
Epoch 2/10
9s - loss: 0.3216 - acc: 0.8584 - val_loss: 0.3114 - val_acc: 0.8912
Epoch 3/10
9s - loss: 0.3514 - acc: 0.8547 - val_loss: 0.2595 - val_acc: 0.9062
Epoch 4/10
9s - loss: 0.2954 - acc: 0.8700 - val_loss: 0.2157 - val_acc: 0.9225
Epoch 5/10
9s - loss: 0.3279 - acc: 0.8638 - val_loss: 0.2202 - val_acc: 0.9163
Epoch 6/10
9s - loss: 0.3171 - acc: 0.8644 - val_l

In [ ]:
model.load_weights('weather.h5') 
img_path = PHOTO_PATH+"/"+"katkam-20160803060000.jpg"
img = image.load_img(img_path , target_size=IMSIZE)
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)

# x = inception.preprocess_input(x)
# x = image.img_to_array(img)
# x = img_datagen.random_transform(x)
x = img_datagen.standardize(x)
preds = model.predict(x)
print('Predicted:', preds)
# [Clear, Cloudy, Fog, Rain, Snow]

In [ ]:
cleaned_data.loc[cleaned_data["Mapped"].str.contains("Fog")]
# cleaned_data["Mapped"].str.contains("Rain")